In [2]:
%pip install --use-feature=2020-resolver autokeras

Defaulting to user installation because normal site-packages is not writeable
  Using cached autokeras-1.0.19-py3-none-any.whl (162 kB)
  Using cached tensorflow-2.8.0-cp38-cp38-manylinux2010_x86_64.whl (497.6 MB)
  Using cached keras_tuner-1.1.2-py3-none-any.whl (133 kB)


You should consider upgrading via the '/share/apps/anaconda3/2020.07/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sklearn.datasets
import sklearn.model_selection
import autokeras as ak
import timeit
import pandas as pd

In [2]:
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id = name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  return (X_train, X_test, y_train, y_test)

In [3]:
X_train, X_test, y_train, y_test = dataSetup(1596)
X_train = pd.DataFrame(X_train).to_numpy()
X_test = pd.DataFrame(X_test).to_numpy()
y_train = pd.DataFrame(y_train).to_numpy()
y_test = pd.DataFrame(y_test).to_numpy()
X_train.shape


(435759, 54)

In [4]:
X_train

array([[ 212,  218,   25, ...,    0,    0,    0],
       [  61,    4,    1, ...,    0,    0,    0],
       [ 274,   15,    8, ...,    0,    0,    0],
       ...,
       [ 634,  327,   12, ...,    0,    0,    0],
       [1339,   81,   29, ...,    0,    0,    0],
       [ 129,  280,   15, ...,    0,    0,    0]])

In [5]:
import numpy as np
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6])

In [6]:
start = timeit.default_timer()
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=1
)
clf.fit(X_train, y_train, epochs=100)
stop = timeit.default_timer()
predicted_y = clf.predict(X_test)
print(clf.evaluate(X_test, y_test))


Trial 1 Complete [00h 24m 10s]
val_accuracy: 0.7355915307998657

Best val_accuracy So Far: 0.7355915307998657
Total elapsed time: 00h 24m 10s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
13618/13618 [==============================] - 23s 2ms/step - loss: 0.6739 - accuracy: 0.6943
Epoch 2/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.6237 - accuracy: 0.7148
Epoch 3/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.6107 - accuracy: 0.7206
Epoch 4/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.6027 - accuracy: 0.7240
Epoch 5/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.5972 - accuracy: 0.7261
Epoch 6/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.5933 - accuracy: 0.7280
Epoch 7/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.5898 - accuracy: 0.7294
Epoch 8/100
13618/13618 [==============================] - 22s 2ms/step - loss

Epoch 76/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.5593 - accuracy: 0.7439
Epoch 77/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.5591 - accuracy: 0.7441
Epoch 78/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.5591 - accuracy: 0.7443
Epoch 79/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.5590 - accuracy: 0.7443
Epoch 80/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.5589 - accuracy: 0.7443
Epoch 81/100
13618/13618 [==============================] - 21s 2ms/step - loss: 0.5589 - accuracy: 0.7444
Epoch 82/100
13618/13618 [==============================] - 21s 2ms/step - loss: 0.5588 - accuracy: 0.7445
Epoch 83/100
13618/13618 [==============================] - 22s 2ms/step - loss: 0.5587 - accuracy: 0.7442
Epoch 84/100
13618/13618 [==============================] - 21s 2ms/step - loss: 0.5586 - accuracy: 0.7444
Epoch 85/100
13618/13618 [===========

In [7]:
print(stop-start)

4172.665033178404


In [8]:
model = clf.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 54)]              0         
                                                                 
 multi_category_encoding (Mu  (None, 54)               0         
 ltiCategoryEncoding)                                            
                                                                 
 normalization (Normalizatio  (None, 54)               109       
 n)                                                              
                                                                 
 dense (Dense)               (None, 32)                1760      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  